In [1]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [2]:
def get_image_stats(image_path):
    with Image.open(image_path) as img:
        return img.size, img.mode

base_path = '../data/raw/Furniture_Data'

image_stats = []
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.lower().endswith('.jpg'):
            image_path = os.path.join(root, file)
            image_stats.append(get_image_stats(image_path))  #So, image_stats is a list of tuples (image_size, image_mode)

In [3]:
print(f"Total images: {len(image_stats)}")
print("First 5 image stats:", image_stats[:5])

Total images: 90082
First 5 image stats: [((350, 350), 'RGB'), ((350, 350), 'RGB'), ((350, 350), 'RGB'), ((350, 350), 'RGB'), ((350, 350), 'RGB')]


### Image Resolution:

In [4]:
def check_resolution(image_path):
    with Image.open(image_path) as img:
        return img.size

resolutions = [check_resolution(os.path.join(root, file)) 
               for root, dirs, files in os.walk(base_path)  # base_path is 'Furniture_Data' Folder
               for file in files if file.endswith('.jpg')]  # there is no '.png' image file in the dataset
print("Average resolution:", np.mean(resolutions, axis=0))

Average resolution: [340.85819587 340.85759641]
